In [1]:
import sys
import csv
import torch
from PIL import Image
from sentence_transformers import SentenceTransformer, util

# Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')

split = sys.argv[1]



C:\Users\Paulina\PycharmProjects\SieciNeuronoweGrafy\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
# Read in.txt
in_data = []
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/in.tsv", "r") as f:
    for line in f:
        in_data.append(line.split("\t")[0].strip())



In [57]:
# Read captions.txt ids
captions_d = {}
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/captions.tsv", "r") as f:
    for line in f:
        s_line = line.split()
        caption_id = int(s_line[0])
        caption_text = " ".join(s_line[1:])
        captions_d[caption_id] = {"text": caption_text, "score": 0}



In [58]:
captions = [captions_d[key]["text"][:81] for key in captions_d]

In [59]:
tokenizer = CLIPTextTokenizer.from_pretrained('openai/clip-vit-base-patch32')
inputs = tokenizer(texts, truncation=True, max_length=77, padding='max_length', return_tensors='pt')

NameError: name 'CLIPTextTokenizer' is not defined

In [60]:
# Don't truncate the caption
captions = [captions_d[key]["text"] for key in captions_d]

#captions_emb = model.encode(captions, max_length=77)

# Normalize the caption embeddings
#captions_emb = model.encode(captions)
captions_emb = model.encode([caption[:77] for caption in captions])

captions_emb = torch.tensor(captions_emb)
captions_emb = torch.nn.functional.normalize(captions_emb, dim=1)

#captions_emb = torch.nn.functional.normalize(captions_emb, dim=1)

answers = []



In [61]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image, ImageEnhance
import os
import sys
import csv
import random

In [62]:
# Process images in batches for speed
batch_size = 32
for i in range(0, len(in_data), batch_size):
    batch_images = in_data[i:i+batch_size]
    
    batch_images_emb = []
    
    for img_name in batch_images:
        img_path = "C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/pictures/" + img_name
        image = Image.open(img_path)

        # Augmentation: apply random brightness, contrast, and rotation
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))

        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))

        image = image.rotate(random.uniform(-30, 30))
        batch_images_emb.append(model.encode(image))
    
    #batch_images_emb = model.encode([Image.open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/pictures/" + img_name) for img_name in batch_images])
    batch_images_emb = torch.tensor(batch_images_emb)
    batch_images_emb = torch.nn.functional.normalize(batch_images_emb, dim=1)
    # Compute similarity using dot product
    cos_scores = torch.mm(batch_images_emb, captions_emb.t())
    for img_idx, img_scores in enumerate(cos_scores):
        for caption_id, score in enumerate(img_scores):
            captions_d[caption_id+1]["score"] = score.item()
        scores = sorted(captions_d, key=lambda x: captions_d[x]["score"], reverse=True)
        answers.append(scores)




In [63]:
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/out1009_new_new.tsv", "w", newline='') as f_out:
    writer = csv.writer(f_out, delimiter="\t")
    for answer in answers:
        writer.writerow([answer[0]])

In [17]:
from sklearn.metrics import average_precision_score
import numpy as np

# Load model results
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/out1009.tsv", "r") as f:
    model_results = [list(map(int, line.strip().split())) for line in f]

# Load expected results
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/expected.tsv", "r") as f:
    expected_results = [list(map(int, line.strip().split())) for line in f]

# Initialize arrays for y_true and y_score
y_true = np.zeros((len(expected_results), len(captions)), dtype=int)
y_score = np.zeros((len(expected_results), len(captions)), dtype=float)

# Populate y_true and y_score
for i in range(len(expected_results)):
    for j in range(len(captions)):
        if j+1 in expected_results[i]:
            y_true[i, j] = 1
        if j+1 in model_results[i]:
            y_score[i, j] = 1 - model_results[i].index(j+1) / len(captions)  # Assign higher scores for higher ranks

# Calculate MAP
map_value = average_precision_score(y_true, y_score)
print(f'MAP: {map_value}')


MAP: 0.19540809624651165


In [22]:
from sklearn.metrics import average_precision_score
import numpy as np

# Load model results
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/out1009.tsv", "r") as f:
    model_results = [list(map(int, line.strip().split())) for line in f]

# Load expected results
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/expected.tsv", "r") as f:
    expected_results = [list(map(int, line.strip().split())) for line in f]

# Initialize arrays for y_true and y_score
y_true = np.zeros((len(expected_results), len(captions)), dtype=int)
y_score = np.zeros((len(expected_results), len(captions)), dtype=float)

# Populate y_true and y_score
for i in range(len(expected_results)):
    for j in range(len(captions)):
        if j+1 in expected_results[i]:
            y_true[i, j] = 1
        if j+1 in model_results[i]:
            y_score[i, j] = 1 - model_results[i].index(j+1) / len(captions)  # Assign higher scores for higher ranks

# Calculate MAP
map_value = average_precision_score(y_true, y_score)
print(f'MAP: {map_value}')


MAP: 0.19540809624651165


TestA

In [53]:
# Read in.txt
in_data = []
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/test-A/in.tsv", "r") as f:
    for line in f:
        in_data.append(line.split("\t")[0].strip())
        
# Read captions.txt ids
captions_d = {}
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/test-A/captions.tsv", "r") as f:
    for line in f:
        s_line = line.split()
        caption_id = int(s_line[0])
        caption_text = " ".join(s_line[1:])
        captions_d[caption_id] = {"text": caption_text, "score": 0}

captions = [captions_d[key]["text"][:81] for key in captions_d]

# Don't truncate the caption
captions = [captions_d[key]["text"] for key in captions_d]
captions_emb = model.encode([caption[:77] for caption in captions])

captions_emb = torch.tensor(captions_emb)
captions_emb = torch.nn.functional.normalize(captions_emb, dim=1)
answers = []


In [54]:
# Process images in batches for speed
batch_size = 32
for i in range(0, len(in_data), batch_size):
    batch_images = in_data[i:i+batch_size]
    
    batch_images_emb = []
    
    for img_name in batch_images:
        img_path = "C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/test-A/pictures/" + img_name
        image = Image.open(img_path)

        # Augmentation: apply random brightness, contrast, and rotation
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))

        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))

        image = image.rotate(random.uniform(-30, 30))
        batch_images_emb.append(model.encode(image))
    
    #batch_images_emb = model.encode([Image.open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/pictures/" + img_name) for img_name in batch_images])
    batch_images_emb = torch.tensor(batch_images_emb)
    batch_images_emb = torch.nn.functional.normalize(batch_images_emb, dim=1)
    # Compute similarity using dot product
    cos_scores = torch.mm(batch_images_emb, captions_emb.t())
    for img_idx, img_scores in enumerate(cos_scores):
        for caption_id, score in enumerate(img_scores):
            captions_d[caption_id+1]["score"] = score.item()
        scores = sorted(captions_d, key=lambda x: captions_d[x]["score"], reverse=True)
        answers.append(scores)




In [55]:
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/out1009_test-A_new_new.tsv", "w", newline='') as f_out:
    writer = csv.writer(f_out, delimiter="\t")
    for answer in answers:
        writer.writerow([answer[0]])

In [ ]:
TestB

In [40]:
# Read in.txt
captions = []
captions_emb = []
in_data = []
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/test-B/in.tsv", "r") as f:
    for line in f:
        in_data.append(line.split("\t")[0].strip())
        
# Read captions.txt ids
captions_d = {}
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/test-B/captions.tsv", "r") as f:
    for line in f:
        s_line = line.split()
        caption_id = int(s_line[0])
        caption_text = " ".join(s_line[1:])
        captions_d[caption_id] = {"text": caption_text, "score": 0}

#captions = [captions_d[key]["text"][:77] for key in captions_d]

# Don't truncate the caption
captions = [captions_d[key]["text"] for key in captions_d]
captions_emb = model.encode([caption[:72] for caption in captions])

captions_emb = torch.tensor(captions_emb)
captions_emb = torch.nn.functional.normalize(captions_emb, dim=1)
answers = []


In [41]:
# Process images in batches for speed
batch_size = 32
for i in range(0, len(in_data), batch_size):
    batch_images = in_data[i:i+batch_size]
    
    batch_images_emb = []
    
    for img_name in batch_images:
        img_path = "C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/test-B/pictures/" + img_name
        image = Image.open(img_path)

        # Augmentation: apply random brightness, contrast, and rotation
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))

        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(random.uniform(0.5, 1.5))

        image = image.rotate(random.uniform(-30, 30))
        batch_images_emb.append(model.encode(image))
    
    #batch_images_emb = model.encode([Image.open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/pictures/" + img_name) for img_name in batch_images])
    batch_images_emb = torch.tensor(batch_images_emb)
    batch_images_emb = torch.nn.functional.normalize(batch_images_emb, dim=1)
    # Compute similarity using dot product
    cos_scores = torch.mm(batch_images_emb, captions_emb.t())
    for img_idx, img_scores in enumerate(cos_scores):
        for caption_id, score in enumerate(img_scores):
            captions_d[caption_id+1]["score"] = score.item()
        scores = sorted(captions_d, key=lambda x: captions_d[x]["score"], reverse=True)
        answers.append(scores)




In [52]:
with open("C:/Users/Paulina/PycharmProjects/SieciNeuronoweGrafy/cnlps-ticrc-master/train/out1009_test-B_new_new.tsv", "w", newline='') as f_out:
    writer = csv.writer(f_out, delimiter="\t")
    for answer in answers:
        writer.writerow([answer[0]])
        #for label in answer:
            #writer.writerow([label])

In [44]:
print(len(answer))

435


In [46]:
print(len(answers))

435


In [51]:
print(answers)

[[370, 319, 367, 132, 166, 247, 239, 242, 103, 208, 304, 318, 207, 384, 147, 52, 193, 182, 196, 121, 426, 211, 64, 118, 330, 338, 112, 360, 57, 270, 61, 87, 210, 31, 359, 195, 273, 114, 74, 75, 120, 414, 90, 113, 430, 309, 237, 94, 39, 161, 93, 283, 332, 224, 326, 191, 348, 70, 1, 141, 433, 124, 180, 27, 347, 78, 276, 26, 50, 340, 221, 331, 59, 413, 171, 327, 257, 169, 170, 101, 22, 217, 268, 252, 190, 281, 73, 253, 365, 144, 316, 416, 385, 294, 420, 303, 363, 411, 95, 345, 152, 362, 368, 154, 79, 428, 355, 98, 245, 233, 192, 14, 232, 295, 308, 37, 100, 151, 80, 341, 271, 230, 324, 321, 184, 425, 46, 9, 387, 397, 119, 376, 389, 203, 20, 356, 343, 10, 254, 248, 142, 388, 153, 379, 350, 173, 258, 284, 62, 256, 123, 88, 259, 218, 187, 55, 126, 19, 249, 421, 422, 328, 269, 58, 201, 364, 163, 56, 351, 216, 415, 116, 136, 202, 236, 158, 131, 89, 199, 278, 406, 240, 134, 194, 305, 60, 285, 412, 265, 344, 209, 214, 30, 215, 206, 403, 125, 92, 65, 227, 23, 288, 334, 130, 200, 159, 342, 213, 231